In [1]:
import random
from my_dice_generator import get_dice_value
import os
import glob
import pickle

class Player():
    
    def __init__(self, name, log,
                bettor_play_handler,
                dealer_play_handler,
                bet_handler,
                call_handler):
        self.total_value = 0
        self.name = name
        self.score = 0
        self.log = log
        self.bettor_play_handler = bettor_play_handler 
        self.dealer_play_handler = dealer_play_handler
        self.bet_handler = bet_handler
        self.call_handler = call_handler
    
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.__repr__()
    
    def get_penalization(self):
        """
        Tato funkce spocita penalizaci hrace.
        
        Pokud hrace nahaze pres 21, je maximalni penalizace = 100
        """
        self.penalization = 100 if self.total_value > 21 else 21 - self.total_value
        
    def roll_dice(self):
        """
        Tato funkce vrací nahodny hod kostkou - integer 1-6.
        
        TODO: zajistit lepsi nahodny cislo
        """
#         numbers = {key: [] for key in range(1,7)}
#         for number in numbers:
#             filename = os.path.join("data", "images", "{}".format(number),"")
#             files = glob.glob(filename + "*.jpg")
#             numbers[number] = files

#         dice_roll_path = random.choice(numbers[random.randint(1,6)])
#         dice_value = get_dice_value(dice_roll_path)
        return random.randint(1,6)
    
    def bettor_play(self):
        while self.total_value < 21:
            self.total_value += self.roll_dice()
            if not self.bettor_play_handler(self.log, self.total_value):
                break
        self.get_penalization()
            
    def dealer_play(self):
        while self.total_value < 17:
            self.total_value += self.roll_dice()
            if not self.dealer_play_handler(self.log, self.total_value):
                break
        self.get_penalization()
            
    def bet(self):
        return self.bet_handler(self.log, self.total_value)
    
    def call(self, bet_size):
        return self.call_handler(self.log, self.total_value, bet_size)
    
    def cleanup(self):
        self.total_value = 0



In [2]:
import os

def update_score(size, winner, loser):
    winner.score += size
    loser.score -= size
    return winner, loser
    
# def game_round(bettor, dealer): 
#     bettor.bettor_play()
#     bet_size = bettor.bet()
#     dealer.dealer_play()
#     if not dealer.call(bet_size):
#         winner, loser = update_score(1, bettor, dealer)
#         called = 0
#     else:
#         if bettor.penalization < dealer.penalization:
#             winner, loser = update_score(bet_size, bettor, dealer)
#         else:
#             winner, loser = update_score(bet_size, dealer, bettor)
#         called = 1
#     log.append((bettor.name, dealer.name, winner, loser, bet_size, called, bettor.total_value, dealer.total_value))   
#     for player in [bettor, dealer]:
#         player.cleanup()
    
# def simulation(p, n):
#     FILENAME = os.path.join("data", "output.csv")
#     for idx in range(n):
#         game_round(p[0], p[1])
#         p[0], p[1] = p[1], p[0] 
#     print(p[0], p[0].score)
#     print(p[1], p[1].score)
#     lines = [",".join(map(str, entry)) for entry in log]
#     text = "#bettor name, dealer name, winner, loser, bet size, called, bettor value, dealer value\n"
#     text += "\n".join(lines)
#     with open(FILENAME, "w") as f:
#         f.write(text)
        
def simulation(p, n):
    FILENAME = os.path.join("data", "output.csv")
    # vytvoreni souboru
    text = "bettor_name,dealer_name,winner,loser,bet_size,called,bettor_value,dealer_value"
    with open(FILENAME, "w") as f:
        f.write(text)
    # zbytek simulace
    for idx in range(n):
        game_round(p[0], p[1], FILENAME)
        p[0], p[1] = p[1], p[0] 
    print(p[0], p[0].score)
    print(p[1], p[1].score)
    #lines = [",".join(map(str, entry)) for entry in log]

    
        
def write_log_entry(log, FILENAME):
    text = "\n" + ",".join(map(str, log[-1]))
    with open(FILENAME, "a") as f:
        f.write(text)    
    
def game_round(bettor, dealer, FILENAME): 
    bettor.bettor_play()
    bet_size = bettor.bet()
    dealer.dealer_play()
    if not dealer.call(bet_size):
        winner, loser = update_score(1, bettor, dealer)
        called = 0
    else:
        if bettor.penalization < dealer.penalization:
            winner, loser = update_score(bet_size, bettor, dealer)
        else:
            winner, loser = update_score(bet_size, dealer, bettor)
        called = 1
    log.append((bettor.name, dealer.name, winner, loser, bet_size, called, bettor.total_value, dealer.total_value))
    write_log_entry(log, FILENAME)
    data = [winner]
    with open("madra_data.dt", "wb") as f:
        pickle.dump(data, f)
    for player in [bettor, dealer]:
        player.cleanup()
    

## Dummy, better player

In [3]:
def better_bettor_play(log, value):
    if value >= 18:
        return False
    else:
        return True

def better_dealer_play(log, value):
    return True

def better_bet(log, value):
    return random.randint(1, 5)

def better_call(log, value, bet_size):
    if value > 21:
        return False
    else:
        return True
    
    
def dummy_bettor_play(log, value):
    return True

def dummy_dealer_play(log, value):
    return True

def dummy_bet(log, value):
    return 5

def dummy_call(log, value, bet_size):
    return True

## myBot

In [4]:
def myBot_bettor_play(log, value):
    if value <= 18:
        return True
    else:
        return False
def myBot_dealer_play(log, value):
    return True
    """
    return True
    """
def myBot_bet(log, value):
    if value == 21:
        return 5
    elif value == 20:
        return 4
    elif value == 19:
        return 3
    elif value == 18:
        x = random.randint(1, 2)
        if x ==1:
            return 1
        else:
            return 5
    else:
        return 1

def myBot_call(log, value, bet_size):
    if value <= 21:
        return True
    else:
        return False
    

# Adam

In [5]:
def adam_bettor_play(log, value):
    if value >= 16:
        return False
    else:
        return True

def adam_dealer_play(log, value):
    return True

def adam_bet(log, value):
    return random.randint(2, 4)

def adam_call(log, value, bet_size):
    if value > 21:
        return False
    else:
        return True

# Adéla

In [6]:
def adela_bettor_play(log, value):
    if value <= 19:
        return True
    else:
        return False

def adela_dealer_play(log, value):
    return True

def adela_bet(log, value):
    bet_size = 0
    if value <= 16:
        bet_size = 1
    if value <=21:
        bet_size = random.randint(3, 5)
    else:
        if random.randint(1, 100) <= 0:
            bet_size = random.randint(1, 2)
    return bet_size

def adela_call(log, value, bet_size):
    if value <= 12:
            return False
    if value <= 21:
        if bet_size <= value - 21 + 5:
            return True

# Tomáš

In [7]:
def better1_bettor_play(log, value):
    if value >= 19:
        return False
    else:
        return True

def better1_dealer_play(log, value):
     return True

def better1_bet(log, value):
    if value == 21:
        return 5
    elif value == 20:
        return random.randint(4, 5)
    elif value == 19:
        return random.randint(3, 4)
    elif value == 18:
        return random.randint(2, 3)
    elif value == 17:
        return random.randint(1, 2)
    else:
        return 0
                
def better1_call(log, value, bet_size):
    if value > 21:
        return False
    else:
        return True

# Filip

In [8]:
def filip_bettor_play(log, value):
    if value < 16:
        return True
    if value < 18:
        return True
    return False

def filip_dealer_play(log, value):
    if value < 17:
        return True
    return False

def filip_bet(log, value):
    x = value
    y = 1
    if x > 21:
        z = random.randint(1, 2)
        if z == 1:
            return 5
        else:
            return 1
    if x < 16:
        y = random.randint(1, 3)
        return y
    if x > 19:
        y = random.randint(4, 5)
        return y
    if x < 19:
        y = random.randint(3, 5)
        return y
    y = 4
    return y

def filip_call(log, value, bet_size):
    x = value
    y = bet_size
    my_strength = 0
    if x > 21:
        return False
    if x > 19:
        my_strength = 3
    elif x > 17:
        my_strength = 2
    elif x > 15:
        my_strength = 1
    else:
        return False
   
    enemy_strength = 0
    if y > 4:
        guess_bluff = random.randint(1, 2)
        if guess_bluff == 2:
            enemy_strength = 1
        else:
            enemy_strength = 3
    elif y > 2:
        enemy_strength = 2
    else:
        enemy_strength = 1
    if my_strength > enemy_strength - 1:
        return True
    return False

# Michal - Gary

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import pickle

def Gary_bettor_play(log, total):
    return True if total < 19 else False

def Gary_dealer_play(log, total):
    return True if total < 19 else False

def Gary_bet(log, total):
        if total == 21:
            return random.randint(4,5)
        elif total == 20:
            return random.randint(3,5)
        elif total == 19:
            return random.randint(3,4)
        else:
            return random.randint(0,1)

def Gary_call(log, total, bet):
    def analyze_opp_bet_v2():
        '''
        Vypise analyzu value v zavislosti na bet size
        vraci dictionary {bet:(median, mean)}
        '''

        #load dat, vyber opponenta jako bettora
        data = pd.read_csv('data/output.csv', delimiter = ',')
        opp_bettor = data.loc[data['bettor_name'] != 'Gary']
        #dictionary pro ulozeni useful hodnot
        expect = {}
        #vypocet a vypis medianu, meanu a variance value pro jednotlive bety 1 - 5
        for idx in range(0,6):
            opp_bettor_customBet = opp_bettor.loc[opp_bettor['bet_size'] == idx]
            #novy list s prepsanim vsech values vyssich nez 21 na 22 pro snazsi analyzu
            mapped_values_list = list(map(lambda x: x if x < 22 else 22, opp_bettor_customBet['bettor_value']))
            #pokud je list prazdny, oponent tuto hodnotu nesazi a nic nepocitam
            if len(mapped_values_list) == 0:
                continue
            #vypocet a vypis medianu, meanu a variance value
            median = statistics.median(mapped_values_list)
            mean = statistics.mean(mapped_values_list)
            varr = np.var(mapped_values_list)
            #pokud neni variance prilis velka, zapise si bet do analyzy
            if varr < 1:
                expect[idx] = (median, mean)
        #vrati analyzu
        return expect

    #udela analyzu vzdy po 1000 kolech
    analysis = {}
    if (len(log) % 1000 == 0) and (len(log) != 0):
    #if len(log) > 100:
        analysis_save = analyze_opp_bet_v2()
        with open('Gary_analysis.pcl', 'wb') as f:
            pickle.dump(analysis_save, f)
    if len(log) > 1000:
        with open('Gary_analysis.pcl', 'rb') as f:
            analysis = pickle.load(f)
    #pokud ma bet v analyze, rozhoduje se
    if bet in analysis.keys():
        #pokud oponent prehazel, call
        if (analysis[bet][0] == 22) or (analysis[bet][1] >= 21.3):
            return True
        #pokud sam prehazel a oponent ne, NO call
        elif total > 21:
            return False
        #pokud ma vic, vsadi
        else:
            return True if total >= max(analysis[bet]) else False
    #pokud nema bet v analyze, sazi kdyz neprehazel
    else:
        return False if total > 21 else True

# Julie

In [22]:
def julie_bettor_play(FILENAME, value):
    if value >= 18:
        return False
    else:
        return True

def julie_dealer_play(FILENAME, value):
    return True


def julie_bet(FILENAME, value):
    try:
        with open("data/output.csv", 'r') as f:
            lines = f.read().splitlines()
            line = lines[-2]
            sliced_line = line.split(",")

        if value == 18:
            return 1
        elif sliced_line[2] == "julie":
            if value > 20:
                return int(sliced_line[4])
            else:
                return random.randint(1, int(sliced_line[4]))
        elif not sliced_line[2] == "julie":
            if int(sliced_line[4]) < 4:
                return int(sliced_line[4]) + 2
            elif int(sliced_line[4]) == 4:
                return int(sliced_line[4]) + 1
            else:
                return int(sliced_line[4])
        else:
            if value == 21 or value == 20:
                return 5
            elif value == 19:
                return random.randint(2, 4)
            else:
                return 1
    except:
        return random.randint(1, 5)
   
   
def julie_call(FILENAME, value, bet_size):
    if value > 21:
        return False
    else:
        if bet_size <= value - 21 + 5:
            return True 

# Martin

In [23]:
def madra_bettor_play(log, value):
    global rating
        
    rate_round()
    rate_position_move()
    rating_update()
        
    if value >= (16 + 3 * rating):
        return False
    else:
        return True

def madra_dealer_play(log, value):
    rate_round()
    rate_position_move()
    rating_update()
    return True

def madra_bet(log, value):
    
    if value > 18 and value < 22:
        return 5
    elif value > 21:
        bluff_chance = random.randint(1, 7)
        if bluff_chance > 6:
            return random.randint(1, 5)
        else:
            return 1
    else:
        return random.randint(1, 5)

def madra_call(log, value, bet_size):
    if (value > 21 or bet_size == 5 and value < 20):
        return False
    else:
        return True

def rate_position_move():
   
    global rate_position, rate_range
   
    rate_position += 1
    if(rate_position > rate_range - 1):
        rate_position = 0
        
def rate_round():
   
    global rate_position, rate, rate_round_count
   
    rate_round_count += 1
    if rate_round_count > 10:

        with open("madra_data.dt", "rb") as f:
            data = pickle.load(f)
           
      
        if(str(data[0]) == "Madra"):
            rate[rate_position] = 1
        else:
            rate[rate_position] = 0
            
def rating_update():
    global rate_range, rate, rating
    rating = (sum(rate) / rate_range)


# Alex

In [24]:
def alex_bettor_handler(log, value):
    return value <= 18

def alex_dealer_handler(log, value):
    return True

def alex_bet_handler(log, value):
    #tohle dokaze porazit ukazkoveho explo2 tak to zkousim
    return 5

def alex_check_handler(log, value, bet):
    #jakykoliv pokus o kreativitu v teto funkci naopak explo2 dokazal brutalne potlacit
    return value <= 21 and bet <= value - 21 + 5

# Markéta

In [25]:
def nice_bettor_play(log, value):
    if value >= 18:
        return False
    else:
        return True

def nice_dealer_play(log, value):
    return True

def nice_bet(log, value):
    
    if not log:
        name = " "
    else:
        name = log[-1][0]

    data = pd.read_csv("data/output.csv",delimiter = ",")
    
    winner_is_B = data.loc[data["winner"] == name]
   
    bettor_is_B = winner_is_B.loc[winner_is_B["bettor_name"] == name]
    #bettor_value_is = winner_is_B.loc[winner_is_B[] == and bettor_is_B["bettor_value"] ==]
       
    if value == 21:
        return 5
    elif value == 18:
        return 1
    elif bettor_is_B["bet_size"].corr(bettor_is_B["bettor_value"])>0.03:
         if value == 19:
            return 2
         else:
            return 4  
    else:
        return 1
    
def nice_call(log, value, bet_size):
    if value < 22:
            if bet_size <= value - 21 + 5:
                return True

# Simulace

In [31]:
rate_position = 0
rate_range = 30
rate = [1 for n in range(rate_range)]
rating = 0
rate_round_count = 0

log = []  


player_julia = Player("Julia", log,
        myBot_bettor_play,
        myBot_dealer_play,
        myBot_bet,
        myBot_call)

better_player = Player("better_player", log,
        better_bettor_play,
        better_dealer_play,
        better_bet,
        better_call)

dummy_player = Player("dummy_player", log,
        dummy_bettor_play,
        dummy_dealer_play,
        dummy_bet,
        dummy_call)

player_adam = Player("Adam", log,
        adam_bettor_play,
        adam_dealer_play,
        adam_bet,
        adam_call)

player_adela = Player("Adela", log,
        adela_bettor_play,
        adela_dealer_play,
        adela_bet,
        adela_call)

player_tomas = Player("Tomas", log,
        better1_bettor_play,
        better1_dealer_play,
        better1_bet,
        better1_call)

player_filip = Player("filip", log,
    filip_bettor_play,
    filip_dealer_play,
    filip_bet,
    filip_call),

player_gary = Player("Gary", log,
    Gary_bettor_play,
    Gary_dealer_play,
    Gary_bet,
    Gary_call)

player_julie = Player("Julie", log,
        julie_bettor_play,
        julie_dealer_play,
        julie_bet,
        julie_call)

player_martin = Player("Madra", log,
        madra_bettor_play,
        madra_dealer_play,
        madra_bet,
        madra_call)

player_alex = Player("Alex", log,
        alex_bettor_handler,
        alex_dealer_handler,
        alex_bet_handler,
        alex_check_handler)

player_marketa = Player("Marketa", log,
        nice_bettor_play,
        nice_dealer_play,
        nice_bet,
        nice_call)

all_players = [
    player_adela,
    player_tomas,
    player_julia,
    player_adam,
    better_player,
    dummy_player,
    player_filip,
    player_gary,
    player_julie,
    player_martin,
    player_alex,
    player_marketa,
]

 

p = [player_martin, player_alex]


simulation(p, 3000)

Madra 240
Alex -240


In [34]:
import itertools

names = {str(p): 0 for p in all_players}
N = 1000

for n1, n2 in itertools.combinations(names, 2):
    player_julia = Player("Julia", log,
            myBot_bettor_play,
            myBot_dealer_play,
            myBot_bet,
            myBot_call)

    better_player = Player("better_player", log,
            better_bettor_play,
            better_dealer_play,
            better_bet,
            better_call)

    dummy_player = Player("dummy_player", log,
            dummy_bettor_play,
            dummy_dealer_play,
            dummy_bet,
            dummy_call)

    player_adam = Player("Adam", log,
            adam_bettor_play,
            adam_dealer_play,
            adam_bet,
            adam_call)

    player_adela = Player("Adela", log,
            adela_bettor_play,
            adela_dealer_play,
            adela_bet,
            adela_call)

    player_tomas = Player("Tomas", log,
            better1_bettor_play,
            better1_dealer_play,
            better1_bet,
            better1_call)
    
    player_filip = Player("filip", log,
        filip_bettor_play,
        filip_dealer_play,
        filip_bet,
        filip_call)
    
    player_gary = Player("Gary", log,
        Gary_bettor_play,
        Gary_dealer_play,
        Gary_bet,
        Gary_call)

    player_julie = Player("Julie", log,
            julie_bettor_play,
            julie_dealer_play,
            julie_bet,
            julie_call)

    player_martin = Player("Madra", log,
            madra_bettor_play,
            madra_dealer_play,
            madra_bet,
            madra_call)

    player_alex = Player("Alex", log,
            alex_bettor_handler,
            alex_dealer_handler,
            alex_bet_handler,
            alex_check_handler)

    player_marketa = Player("Marketa", log,
            nice_bettor_play,
            nice_dealer_play,
            nice_bet,
            nice_call)

    all_players = [
        player_adela,
        player_tomas,
        player_julia,
        player_adam,
        better_player,
        dummy_player,
        player_filip,
        player_gary,
        player_julie,
        player_martin,
        player_alex,
        player_marketa,
    ]
    
    log = []
    
    p1 = [p for p in all_players if str(p) == n1]
    p2 = [p for p in all_players if str(p) == n2]
    p = p1 + p2
    
    simulation(p, N)
    
    if p1[0].score > 0:
        names[str(p1[0])] += 1
    else:
        names[str(p2[0])] += 1
        
print("\n\n\nVýsledky:\n")
names = sorted(names.items(), key=lambda kv: kv[1], reverse=True)

for k, v in names:
    print(k, "\t", v)
    

Adela 643
Tomas -643
Adela 860
Julia -860
Adela 971
Adam -971
Adela 746
better_player -746
Adela 347
dummy_player -347
Adela 359
filip -359
Adela 585
Gary -585
Adela 51
Julie -51
Adela 310
Madra -310
Adela -48
Alex 48
Adela 115
Marketa -115
Tomas 6
Julia -6
Tomas 1388
Adam -1388
Tomas 782
better_player -782
Tomas 1933
dummy_player -1933
Tomas 113
filip -113
Tomas 344
Gary -344
Tomas -207
Julie 207
Tomas 245
Madra -245
Tomas 121
Alex -121
Tomas -273
Marketa 273
Julia 1120
Adam -1120
Julia 435
better_player -435
Julia 1604
dummy_player -1604
Julia 85
filip -85
Julia -148
Gary 148
Julia -346
Julie 346
Julia 118
Madra -118
Julia -387
Alex 387
Julia -424
Marketa 424
Adam -714
better_player 714
Adam 337
dummy_player -337
Adam -766
filip 766
Adam -1159
Gary 1159
Adam -409
Julie 409
Adam -844
Madra 844
Adam -394
Alex 394
Adam -549
Marketa 549
better_player 1157
dummy_player -1157
better_player 19
filip -19
better_player -471
Gary 471
better_player -175
Julie 175
better_player -159
Madra 159
be